# Tutorial: Using the DatabaseManager (DragonSQL)

This notebook demonstrates how to use the `DragonSQL` from the `dragon-ml-toolbox`. This utility provides a simple and powerful interface for interacting with SQLite databases, abstracting away the boilerplate code for connections, cursors, and data handling.

Workflow:

1. Connecting to a database.
2. Creating tables with a specific schema.
3. Inspecting the database structure.
4. Inserting data (single rows, bulk rows, and from pandas DataFrames).
5. Querying data back into a DataFrame.
6. Optimizing query performance by creating indexes.
7. Executing custom SQL commands.

### **Cell 1: Setup and Imports**

In [ ]:
import pandas as pd
from pathlib import Path
import os
from ml_tools.SQL import DragonSQL

# Define the path for the database
DB_PATH = Path("experiment_results.db")

# Clean up previous runs
if DB_PATH.exists():
    os.remove(DB_PATH)
    print(f"Removed old database: '{DB_PATH}'")

print("Setup complete. Ready to start.")

### **Cell 2: Connecting and Creating a Table**

The `DatabaseManager` is best used as a context manager (`with` statement), which automatically handles opening and closing the connection, as well as committing changes or rolling them back if an error occurs.

Let's define a schema for a table that will store results from a fictional experiment.

In [ ]:
# Define the schema for our results table
experiment_schema = {
    "result_id": "INTEGER PRIMARY KEY AUTOINCREMENT",
    "experiment_name": "TEXT NOT NULL",
    "feature_alpha": "REAL",
    "feature_beta": "REAL",
    "is_priority": "INTEGER", # 0 for False, 1 for True
    "score": "REAL"
}

# Connect to the database and create the table
try:
    with DragonSQL(DB_PATH) as db:
        print("--- Creating table 'results' ---")
        db.create_table("results", experiment_schema)
        print("Table 'results' created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 3: Inspecting the Database**

Now that we've created a table, let's verify it exists and check its schema using the built-in helper methods.

In [ ]:
try:
    with DragonSQL(DB_PATH) as db:
        print("--- Listing all tables in the database ---")
        tables = db.list_tables()
        print(f"Tables found: {tables}")

        print("\n--- Getting the schema for the 'results' table ---")
        schema_df = db.get_table_schema("results")
        print(schema_df)
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 4: Inserting Data - One Row at a Time**

The `insert_row` method is perfect for inserting single records, which is useful for logging events or results as they happen.

In [ ]:
# Data for our first result
result_one = {
    "experiment_name": "alpha_test",
    "feature_alpha": 0.15,
    "feature_beta": 0.85,
    "is_priority": 0,
    "score": 92.3
}

try:
    with DragonSQL(DB_PATH) as db:
        print("--- Inserting a single row ---")
        db.insert_row("results", result_one)
        
        # Let's verify by querying the data
        df = db.query_to_dataframe("SELECT * FROM results")
        print(df)
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 5: Inserting Data - Bulk Insertion**

For performance, when you have many records to insert at once, `insert_many` is far more efficient than calling `insert_row` in a loop.

In [ ]:
# A list of dictionaries for bulk insertion
many_results = [
    {"experiment_name": "beta_test", "feature_alpha": 0.5, "feature_beta": 0.5, "is_priority": 1, "score": 95.1},
    {"experiment_name": "beta_test", "feature_alpha": 0.4, "feature_beta": 0.6, "is_priority": 0, "score": 94.8},
    {"experiment_name": "beta_test", "feature_alpha": 0.3, "feature_beta": 0.7, "is_priority": 0, "score": 94.5},
]

try:
    with DragonSQL(DB_PATH) as db:
        print("--- Inserting multiple rows with insert_many ---")
        db.insert_many("results", many_results)
        
        # Verify the new data
        df = db.query_to_dataframe("SELECT * FROM results")
        print(df)
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 6: Inserting Data - From a Pandas DataFrame**

A very common use case is having data already in a pandas DataFrame. The `insert_from_dataframe` method handles this seamlessly.

In [ ]:
# Create a sample DataFrame
df_results = pd.DataFrame([
    {"experiment_name": "gamma_run", "feature_alpha": 0.9, "feature_beta": 0.1, "is_priority": 1, "score": 99.8},
    {"experiment_name": "gamma_run", "feature_alpha": 0.8, "feature_beta": 0.2, "is_priority": 1, "score": 99.1},
])

try:
    with DragonSQL(DB_PATH) as db:
        print("--- Inserting data from a pandas DataFrame ---")
        db.insert_from_dataframe("results", df_results)
        
        # Verify the final state of the table
        df = db.query_to_dataframe("SELECT * FROM results")
        print("--- Full table content ---")
        print(df)
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 7: Querying Data**

The `query_to_dataframe` method can execute any `SELECT` statement. For security and correctness, it's best practice to use `?` placeholders for parameters.

In [ ]:
try:
    with DragonSQL(DB_PATH) as db:
        print("--- Querying all 'beta_test' experiments ---")
        beta_df = db.query_to_dataframe(
            "SELECT * FROM results WHERE experiment_name = ?", 
            ("beta_test",)
        )
        print(beta_df)

        print("\n--- Querying high-priority runs with a score > 99 ---")
        high_score_df = db.query_to_dataframe(
            "SELECT experiment_name, score FROM results WHERE is_priority = ? AND score > ?",
            (1, 99.0)
        )
        print(high_score_df)
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 8: Creating an Index for Performance**

When a table grows large, queries can become slow. Creating an index on frequently queried columns dramatically speeds up data retrieval.

In [ ]:
try:
    with DragonSQL(DB_PATH) as db:
        print("--- Creating an index on the 'score' column ---")
        db.create_index("results", "score")
        
        print("\n--- Creating a second index on 'experiment_name' ---")
        db.create_index("results", "experiment_name")
        
        print("\nIndexes created. Queries on 'score' and 'experiment_name' will now be much faster.")
except Exception as e:
    print(f"An error occurred: {e}")

### **Cell 9: Executing Custom SQL**

For any other SQL command that doesn't return data (like `UPDATE` or `DELETE`), you can use the `execute_sql` method.

In [ ]:
try:
    with DragonSQL(DB_PATH) as db:
        print("--- Updating a record using execute_sql ---")
        # Let's update the score for result_id = 1
        db.execute_sql("UPDATE results SET score = ? WHERE result_id = ?", (92.5, 1))
        
        # Verify the change
        updated_row = db.query_to_dataframe("SELECT * FROM results WHERE result_id = 1")
        print(updated_row)
except Exception as e:
    print(f"An error occurred: {e}")